In [2]:
!pip install scienceplots
!pip install wget

  Using cached SciencePlots-2.1.1-py3-none-any.whl.metadata (11 kB)
Using cached SciencePlots-2.1.1-py3-none-any.whl (16 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a23959a36dd6a509cd889517a53b70c3235da128523f3dc92696be8460fd90f3
  Stored in directory: /Users/phurailatpamhemantakumar/Library/Caches/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [5]:
# Downlaod GRB250313A light curve data (n0-n5) from Fermi GBM FTP server (note: Also n6, n7, n8, n9, na, nb available, but not used here)
# The data is in FITS format, which is a standard format for astronomical data
# Different n0, n1, n2, n3, n4, n5 correspond to different energy channels between ~8 keV and 1 MeV (maybe, according to CHATGPT lol)
import wget
filenames = ['glg_tte_n0_bn250313607_v00.fit', 
            'glg_tte_n1_bn250313607_v00.fit', 
            'glg_tte_n2_bn250313607_v00.fit', 
            'glg_tte_n3_bn250313607_v00.fit', 
            'glg_tte_n4_bn250313607_v00.fit', 
            'glg_tte_n5_bn250313607_v00.fit']
urls = ['https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2025/bn250313607/current/' + filename for filename in filenames]
for url in urls:
    wget.download(url) # download the file

In [6]:
# Import necessary libraries; mainly astropy is used to read the fits files
import scienceplots
import pylab as plt
plt.style.use(['science','ieee'])
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# Plot the light curves for each of the 6 energy channels
fig, ax = plt.subplots(6, 1, figsize=(15, 5*5)) # 6 rows, 1 column
# Make one figure for each energy channel
for i, filename in enumerate(filenames):
    # Open the FITS file
    with fits.open(filename) as hdul:
        data = hdul['EVENTS'].data
        photon_arrival_times = data['TIME']  # photon arrival times
    # Bin the data to make a light curve
    bin_width = 0.1 # seconds (change this to change the bin width) ; mainly for visualisation
    t_min = photon_arrival_times.min()
    t_max = photon_arrival_times.max()
    bins = np.arange(t_min, t_max + bin_width, bin_width)
    # Plot the light curve
    ax[i].hist(photon_arrival_times, bins=bins, color='C0', histtype='step', lw=2)
    ax[i].set_xlabel('Time (s)')
    ax[i].set_ylabel(f'Counts per {bin_width:.1f}s')
    ax[i].set_title(f'Light Curve: {filename}')
    # ax[i].tight_layout()
    ax[i].set_xlim(t_min, t_max)
    ax[i].set_ylim(0, 750)
plt.savefig('grb250313A_example_lightcurves.pdf', bbox_inches='tight')
plt.show()